In [ ]:
pip install pyspark-ai

In [11]:
from pyspark_ai import SparkAI
from google.colab import userdata

import os
os.environ["OPENAI_API_KEY"] = userdata.get('openai_key')

spark_ai = SparkAI()
spark_ai.activate()

In [12]:
# Generate sample data
import numpy as np
import pandas as pd

accounts = ['ACC123456', 'ACC234567', 'ACC345678', 'ACC456789', 'ACC567890']
merchants = ['ABC Store', 'ATM', 'Employer', 'Stock Exchange', 'XYZ Electronics', 'Online Store', 'Cafe', 'Supermarket']

np.random.seed(0)
data = {
    'Transaction ID': np.arange(1000000000, 1000000000 + 50),
    'Account ID': np.random.choice(accounts, 50),
    'Transaction Amount': np.round(np.random.uniform(10.00, 2000.00, 50), 2),
    'Transaction Type': np.random.choice(['Purchase', 'Withdrawal', 'Deposit', 'Trade'], 50),
    'Merchant/Counterparty': np.random.choice(merchants, 50),
    'Location': np.random.choice(['New York, NY', 'Los Angeles, CA', 'Chicago, IL', 'Miami, FL', 'Online'], 50),
    'Date and Time': pd.date_range(start='2024-08-01', periods=50, freq='H').strftime('%Y-%m-%d %H:%M').tolist()
}

# Create Spark DataFrame
pdf = pd.DataFrame(data)
df = spark_ai._spark.createDataFrame(pdf)

<ipython-input-12-c4e4909317f0>:16: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  'Date and Time': pd.date_range(start='2024-08-01', periods=50, freq='H').strftime('%Y-%m-%d %H:%M').tolist()


In [13]:
df.ai.transform("For each account, find the merchant with the highest total transaction amount and show the amount + name of merchant").show()

INFO: Creating temp view for the transform:
df.createOrReplaceTempView("spark_ai_temp_view__1723280402")



INFO:spark_ai:Creating temp view for the transform:
df.createOrReplaceTempView("spark_ai_temp_view__1723280402")





> Entering new AgentExecutor chain...
Thought: I will first group by Account ID and Merchant/Counterparty and sum the Transaction Amount. Then, I will use a window function to get the merchant with the highest total transaction amount for each account.
Action: query_validation
Action Input: SELECT `Account ID`, `Merchant/Counterparty`, SUM(`Transaction Amount`) as TotalAmount FROM `spark_ai_temp_view__1723280402` GROUP BY `Account ID`, `Merchant/Counterparty`
Observation: OK
Thought:I will now use a window function to get the merchant with the highest total transaction amount for each account.
Final Answer: SELECT `Account ID`, `Merchant/Counterparty`, TotalAmount FROM (SELECT `Account ID`, `Merchant/Counterparty`, TotalAmount, ROW_NUMBER() OVER (PARTITION BY `Account ID` ORDER BY TotalAmount DESC) as rn FROM (SELECT `Account ID`, `Merchant/Counterparty`, SUM(`Transaction Amount`) as TotalAmount FROM `spark_ai_temp_view__1723280402` GROUP BY `Account ID`, `Merchant/Counterparty`)) WH